In [1]:
#Python packages
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 400)
import re
from scipy import stats

from meteocalc import Temp, dew_point, heat_index, wind_chill, feels_like
import pvlib

#Visualization packages
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#ML modeling packages
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

## Competition Overview

Significant investments are being made to improve building efficiencies to reduce costs and emissions. The question is, are the improvements working? That’s where you come in. Under pay-for-performance financing, the building owner makes payments based on the difference between their real energy consumption and what they would have used without any retrofits. The latter values have to come from a model. Current methods of estimation are fragmented and do not scale well. Some assume a specific meter type or don’t work with different building types.

In this competition, you’ll develop accurate models of metered building energy usage in the following areas: chilled water, electric, hot water, and steam meters. The data comes from over 1,000 buildings over a three-year timeframe. With better estimates of these energy-saving investments, large scale investors and financial institutions will be more inclined to invest in this area to enable progress in building efficiencies.

In [2]:
train = pd.read_csv('train_fe.csv')
# test = pd.read_csv('test_reduced.csv')

In [3]:
train.rename({"(True, 'hour_roc')":'hourly_roc'}, axis=1, inplace=True)

In [4]:
train.drop("(False, 'hour_roc')", axis=1, inplace=True)

In [5]:
train = train.iloc[:,1:]

In [6]:
train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,weekday_name,weekday,month,day_of_year,air_temperature_f,wind_speed_mph,dew_temp_f,wind_chill,heat_index,feels,precip_cm,radiation,pressure_pas,airmass,altitude,solar_spec,hourly_roc
0,0,Electricity,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,Friday,4,1,1,77.0,0.0,68.0,83.5955,77.596,77.0,NaN,1413.981805,101950.0,101950.0,-51.773303,NaN,1.784073
1,1,Electricity,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,Friday,4,1,1,77.0,0.0,68.0,83.5955,77.596,77.0,NaN,1413.981805,101950.0,101950.0,-51.773303,NaN,1.233420
2,2,Electricity,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,Friday,4,1,1,77.0,0.0,68.0,83.5955,77.596,77.0,NaN,1413.981805,101950.0,101950.0,-51.773303,NaN,0.940957
3,3,Electricity,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,Friday,4,1,1,77.0,0.0,68.0,83.5955,77.596,77.0,NaN,1413.981805,101950.0,101950.0,-51.773303,NaN,0.715433
4,4,Electricity,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,Friday,4,1,1,77.0,0.0,68.0,83.5955,77.596,77.0,NaN,1413.981805,101950.0,101950.0,-51.773303,NaN,0.836235


In [6]:
'''Variable Description'''
def description(df):
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.iloc[0].values
    summary['Second Value'] = df.iloc[1].values
    summary['Third Value'] = df.iloc[2].values
    return summary

In [7]:
description(train)

,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value
0,building_id,int64,0,1449,0,1,2
1,meter,object,0,4,Electricity,Electricity,Electricity
2,timestamp,object,0,8784,2016-01-01 00:00:00,2016-01-01 00:00:00,2016-01-01 00:00:00
3,meter_reading,float64,0,1745619,0,0,0
4,site_id,int64,0,16,0,0,0
5,primary_use,object,0,16,Education,Education,Education
6,square_feet,int64,0,1397,7432,2720,5376
7,year_built,float64,12127645,116,2008,2004,1991
8,floor_count,float64,16709167,18,NaN,NaN,NaN
9,air_temperature,float64,96658,619,25,25,25


In [6]:
train['random'] = np.random.randint(low=0,high=100000000, size=len(train))

In [7]:
train['age'] = train['year_built'].max() - (train['year_built']+1)

In [8]:
roc_month = train.groupby(['site_id', 'month'])['air_temperature_f'].agg(np.mean).reset_index('month').reset_index()

roc_month['dif']=0.00
for i in range(len(roc_month)-1):
    roc_month['dif'][i] = abs((roc_month['air_temperature_f'][i+1])-(roc_month['air_temperature_f'][i]))

In [17]:
roc_month.reset_index()
roc_month.columns

Index(['site_id', 'month', 'air_temperature_f', 'dif'], dtype='object')

In [14]:
roc_month

,site_id,month,air_temperature_f,dif
0,0,1,61.890324,0.816600
1,0,2,61.073724,9.205219
2,0,3,70.278943,2.100308
3,0,4,72.379251,4.135752
4,0,5,76.515003,4.748615
5,0,6,81.263619,2.131696
6,0,7,83.395315,1.692164
7,0,8,81.703151,1.341237
8,0,9,80.361914,5.105583
9,0,10,75.256331,7.162367


In [18]:
for i in range(len(train['month'].unique())+1):
    train[(train['month'][i]==roc_month['month'][i]) & (train['site_id'][i]==roc_month['site_id'][i]), 'month_roc']=roc_month['dif']

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [19]:
train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,weekday_name,weekday,month,day_of_year,air_temperature_f,wind_speed_mph,dew_temp_f,wind_chill,heat_index,feels,precip_cm,radiation,pressure_pas,airmass,altitude,solar_spec,hourly_roc,random,age,"(True, month_roc)"
0,0,Electricity,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,Friday,4,1,1,77.0,0.0,68.0,83.5955,77.596,77.0,NaN,1413.981805,101950.0,101950.0,-51.773303,NaN,1.784073,24802456,8.0,0.816600
1,1,Electricity,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,Friday,4,1,1,77.0,0.0,68.0,83.5955,77.596,77.0,NaN,1413.981805,101950.0,101950.0,-51.773303,NaN,1.233420,61316517,12.0,9.205219
2,2,Electricity,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,Friday,4,1,1,77.0,0.0,68.0,83.5955,77.596,77.0,NaN,1413.981805,101950.0,101950.0,-51.773303,NaN,0.940957,63831873,25.0,2.100308
3,3,Electricity,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,Friday,4,1,1,77.0,0.0,68.0,83.5955,77.596,77.0,NaN,1413.981805,101950.0,101950.0,-51.773303,NaN,0.715433,56365151,14.0,4.135752
4,4,Electricity,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,Friday,4,1,1,77.0,0.0,68.0,83.5955,77.596,77.0,NaN,1413.981805,101950.0,101950.0,-51.773303,NaN,0.836235,4040317,41.0,4.748615


In [20]:
train.rename({'(True, month_roc)':'month_roc'}, axis=1, inplace=True)

In [21]:
for col in train:
    if train[col].dtype==np.int64:
        train[col] = train[col].astype(np.int16)
    elif train[col].dtype==np.float64:
        train[col] = train[col].astype(np.float16)

In [ ]:
train.to_csv('train_to_encode.csv')

### modeling for feature selection

In [ ]:
from sklearn.pipeline import pipeline
from sklearn import StandardScaler

num_pipeline = Pipeline([
	('imputer', SimpleImputer(strategy='median')),
	('attribs_adder', CombinedAttribAdder()),
	('std_scalar', StandardScaler()),
	])

housing_num = num_pipeline.fit_transform(housing_num)

from sklearn import ColumnTransformer

full_pipeline = ColumnTransformer([
	('num',num_pipeline,num_attribs),
	('cat',OneHotEncoder,cat_attribs),
	])

housing_prepared = full_pipeline.fit_transform(housing)